<a href="https://colab.research.google.com/github/jhmuller/mlp-covid19/blob/main/mlp_covid19_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# need to intall this every time on colab, not sure why but I do it
try:
  import fsspec
except Exception as e:
  print(e)
  !pip install fsspec

In [2]:
import os
import sys
import datetime
import unicodedata
import warnings
import string
import re
import random
import json
import glob
import fsspec
from __future__ import unicode_literals, print_function, division
import re
import nltk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
now = datetime.datetime.now
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def warning_on_one_line(message, category, filename, lineno, file=None, line=None):
    return '\033[1m' +\
    'line:%s  cat:<%s>  msg:%s'% (lineno, category.__name__, message) +\
     '\033[0m' 

warnings.formatwarning = warning_on_one_line

In [3]:
warnings.warn("foo")
# hmm, colors not working, find a fix

line:1  cat:<UserWarning>  msg:foo

In [4]:
# code to show what versions of modules I'm using
import inspect
mlist = list(filter(lambda x: inspect.ismodule(x[1]), locals().items()))
vi = sys.version_info
print("version {0}.{1}.{2} of Python".format(vi.major, vi.minor, vi.micro))
for name, mod in mlist:
    mname = name
    if name.startswith("__"):
      print(name, mod)
      continue
    if hasattr(mod, "__version__"):
        mname = name
        if hasattr(mod, "__path__"):
            mname = os.path.split(mod.__path__[0])[1]
        print("version {1} of {0} as {2} ".format(mname, name, mod.__version__))
    elif hasattr(mod, "__file__") and "site-packages" in mod.__file__:
        print("No __version__ for {0} as {1}".format(mname, name))
del mod
del name

version 3.6.9 of Python
__builtin__ <module 'builtins' (built-in)>
__builtins__ <module 'builtins' (built-in)>
version fsspec of fsspec as 0.8.4 
version re of re as 2.2.1 
version json of json as 2.0.9 
version nltk of nltk as 3.2.5 
version pd of pandas as 1.1.4 
version np of numpy as 1.18.5 
version torch of torch as 1.7.0+cu101 


In [5]:
# to mount google drive for the input files
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
# make a list of all the file paths in the data directory
dpath = "/content/gdrive/My Drive/data/mlp_covid19/16119_db21c91a1ab47385bb13773ed8238c31"
jfiles = os.listdir(dpath)
jpaths = [os.path.join(dpath, jf) for jf in jfiles]
print(jfiles[:4])

['16119_webhose_2020_03_db21c91a1ab47385bb13773ed8238c31_0000015.json', '16119_webhose_2020_03_db21c91a1ab47385bb13773ed8238c31_0000005.json', '16119_webhose_2020_03_db21c91a1ab47385bb13773ed8238c31_0000016.json', '16119_webhose_2020_03_db21c91a1ab47385bb13773ed8238c31_0000023.json']


In [7]:
# the files that we will use here
files_set = set(
    ["16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json",
      "16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json"])

In [8]:
# try reading the data in the first file using the json module
# raised error
f1 = list(files_set)[0]
p1 = os.path.join(dpath, f1)
try:
  with open(p1) as f:
    data = json.load(f)
except Exception as e:
  print(e)
  # sys.exc_info returns (type, value, traceback)
  extype, exval, tb = sys.exc_info()
  warnings.warn(str(extype))

Extra data: line 2 column 1 (char 8133)


line:12  cat:<UserWarning>  msg:<class 'json.decoder.JSONDecodeError'>

In [11]:
# try reading the data with pandas
# raised error
f1 = list(files_set)[0]
p1 = os.path.join(dpath, f1)
try:
  data = pd.read_json(p1)
except Exception as e:
  print(e)   
  # sys.exc_info returns (type, value, traceback)
  extype, exval, tb = sys.exc_info()
  warnings.warn(str(extype)) 

Trailing data


line:11  cat:<UserWarning>  msg:<class 'ValueError'>

In [14]:
# So read in as text 

# only using 2 files
# 16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json 
# 16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json.
files_set = set(
    ["16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json",
      "16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json"])
txt_data = ''
for fpath in jpaths:
  head, tail = os.path.split(fpath)
  if tail in files_set:
    print("reading {0} {1}".format(tail, datetime.datetime.now()))
    with open(jpaths[0], mode='r') as fp:
      new_data = fp.read()
    print("len {0} {1}".format(len(new_data), datetime.datetime.now()))
    txt_data += new_data

print("len of txt data {0}".format(len(txt_data)))
print(now())

reading 16119_webhose_2019_12_db21c91a1ab47385bb13773ed8238c31_0000001.json 2020-12-05 20:32:08.042493
len 521195604 2020-12-05 20:32:11.483658
reading 16119_webhose_2020_01_db21c91a1ab47385bb13773ed8238c31_0000001.json 2020-12-05 20:32:11.483906
len 521195604 2020-12-05 20:32:15.009417
len of txt data 1042391208
2020-12-05 20:32:16.334985


In [15]:
# split into lines
# eval each line
# WARNING: should not do this with untrusted files
txt_lines = txt_data.split('\n')
print("{0} lines".format(len(txt_lines)))
dlist = []
for i, l in enumerate(txt_lines):
  try:
    d = eval(l) 
    dlist.append(d)
  except Exception as e:
    print("line{0}, {1}\n{2}".format(i,l, e))
print("len dlist {0}".format(len(dlist)))

197913 lines
line98956, {"organizations": [], "uuid": "50db3ae27600471c975e2b3f9ffae3693f861967", "thread": {"social": {"gplus": {"shares": 0}, "pinterest": {"shares": 0}, "vk": {"shares": 0}, "linkedin": {"shares": 0}, "facebook": {"likes": 0, "shares": 0, "comments": 0}, "stumbledupon": {"shares": 0}}, "site_full": "www.businessinsider.sg", "main_image": "https://static.businessinsider.sg/sites/2/2020/03/5c24ee842a5b7429c428e9cc.png", "site_section": "https://www.businessinsider.sg/career", "section_title": "Latest Career on Business Insider Singapore - Business Insider Singapore", "url": "https://www.businessinsider.sg/amazon-walmart-hiring-how-to-apply-2020-3", "country": "SG", "domain_rank": 26759, "title": "Amazon and Walmart are ramping up hiring to add 250,000 new jobs. Here's how to apply and whether you can expect an interview.,", "performance_score": 0, "site": "businessinsider.sg", "participants_count": 1, "title_full": "", "spam_score": 0.027, "site_type": "news", "publish

In [30]:
#extract text and title
dataset = [d['text'] for d in dlist]
target = [d['title'] for d in dlist]
# The length of the list dataset and target will be 190138.
print("dataset len= {0}, target len= {1}\n".format(len(dataset), len(target)))
names = ("dataset", "target")
n = 2
for name in names:
  print("first {0} from {1}".format(n,name))
  items = eval(name)[:n]
  maxlen = 30
  for i, item in enumerate(items):
    suffix = "..." if len(item) > maxlen else ""
    print("{0}: {1}{2}".format(i, item[:maxlen], suffix))
  print()

dataset len= 197912, target len= 197912

first 2 from dataset
0: India News COVID-19: Goa SEC c...
1: John Legend: 'The Voice' episo...

first 2 from target
0: COVID-19: Goa SEC cancels Marc...
1: EarthLink - News



In [31]:
contraction_map = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [33]:
# took a litte trial and error to get that
# have to call nltk.download() first
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))  
print(type(stop_words))
print(list(stop_words)[:4])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<class 'set'>
['in', 'where', 'myself', 'own']


In [35]:
# my preprocess funtion
def preprocess(text, contraction_map= None, stop_words = None):
  if not contraction_map:
    contraction_map = {}
  if not stop_words:
    stop_words = []
  if not isinstance(stop_words, set):
    stop_words = set(stop_words)

  # Split the text using Python split() function
  toks = text.split()
  # Apply the contraction hashmap on all the words of the text
  toks = [contraction_map[x] if x in contraction_map.keys() else x for x in toks]  
  # Remove the stopwords that are in English
  toks = [x for x in toks if x not in stop_words]

  # now rejoin to apply other transforms on text string
  res = ' '.join(toks)
  # Convert text to lowercase
  res = res.lower()    
  # Remove 's. For example yours becomes your
  res = res.replace("'s",'') # convert your's -> your
  # Use regular expression to remove parentheses outside a word. For example (word) becomes word
  res = re.sub("\(|\)", "", res) 
  # Use regular expression to remove punctuations
  res = re.sub(r'[^a-zA-Z0-9. ]','',res)
  # Use regular expression to add a space character before and after the full stop. For example . becomes .
  res = re.sub("\.", " . ", res)
  return res
test = "The boys can't .(go) there!."
test_out = preprocess(test, 
           contraction_map=contraction_map, 
           stop_words=stop_words)
print("{0}\n{1}".format(test, test_out))

The boys can't .(go) there!.
the boys cannot  . go there . 


In [36]:
# suggested preprocess function
def preprocess_ref(text, contraction_map, stop_words):
    text = text.lower() # lowercase
    text = text.split() # convert have'nt -> have not
    for i in range(len(text)):
        word = text[i]
        if word in contraction_map:
            text[i] = contraction_map[word]
    text = " ".join(text)
    text = text.split()
    newtext = []
    for word in text:
        if word not in stop_words:
            newtext.append(word)
    text = " ".join(newtext)
    text = text.replace("'s",'') # convert your's -> your
    text = re.sub(r'\(.*\)','',text) # remove (words)
    text = re.sub(r'[^a-zA-Z0-9. ]','',text) # remove punctuations
    text = re.sub(r'\.',' . ',text)
    return text
test = "The boys can't .(go) there!."
test_out = preprocess(test, 
           contraction_map=contraction_map, 
           stop_words=stop_words)
print("{0}\n{1}".format(test, test_out))    

The boys can't .(go) there!.
the boys cannot  . go there . 


In [37]:
%time X = [preprocess(line, contraction_map=contraction_map, stop_words=stop_words) for line in dataset]

CPU times: user 43.7 s, sys: 2.04 s, total: 45.8 s
Wall time: 45.8 s


In [38]:
%time X_ref = [preprocess_ref(line, contraction_map=contraction_map, stop_words=stop_words) for line in dataset]

CPU times: user 48.5 s, sys: 2.71 s, total: 51.2 s
Wall time: 51.2 s


In [39]:
%time Y = [preprocess(line, contraction_map=contraction_map, stop_words=stop_words) for line in target]

CPU times: user 1.68 s, sys: 30.8 ms, total: 1.71 s
Wall time: 1.71 s


In [40]:
%time Y_ref = [preprocess_ref(line, contraction_map=contraction_map, stop_words=stop_words) for line in target]

CPU times: user 1.75 s, sys: 29.8 ms, total: 1.78 s
Wall time: 1.78 s


In [43]:
# At this point, the length of X and Y should both be 190138.
print("len X= {0}, len Y= {1}".format(len(X), len(Y)))
print("len X_ref= {0}, len Y_ref= {1}".format(len(X_ref), len(Y_ref)))
#assert(len(x) == 190138)

len X= 197912, len Y= 197912
len X_ref= 197912, len Y_ref= 197912


In [44]:
# How does this relate to the delivarble?
short_text=[]
short_summary=[]
max_len_text = 600
max_len_target = 30
for i in range(len(dataset)):
    if(len(target[i].split())<=max_len_target and len(dataset[i].split())<=max_len_text):
        short_text.append(dataset[i])
        short_summary.append(target[i])

temp_df=pd.DataFrame({'text':short_text,'summary':short_summary})
print("temp_df shape {0}".format(temp_df.shape))
# len(temp_df) being 130736.

temp_df shape (143468, 2)


In [45]:
temp_df.head(2)

,text,summary
0,India News COVID-19: Goa SEC cancels March 24 ...,COVID-19: Goa SEC cancels March 24 ZP polls; b...
1,John Legend: 'The Voice' episodes taped until ...,EarthLink - News


In [46]:
# remove empty strings from summary and the text column.
newdf = temp_df[temp_df['summary'].str.strip().astype(bool)]
df = newdf[newdf['text'].str.strip().astype(bool)]
print("df shape {0}".format(df.shape))
print(df.head(3))

df shape (130950, 2)
                                                text                                            summary
0  India News COVID-19: Goa SEC cancels March 24 ...  COVID-19: Goa SEC cancels March 24 ZP polls; b...
1  John Legend: 'The Voice' episodes taped until ...                                   EarthLink - News
2  Administrative Leave Offered to County Workers...     Administrative Leave Offered to County Workers


In [47]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [50]:
df.head(2)
X = list(df['text'].values)
Y = list(df['summary'].values)

KeyError: ignored

In [49]:
def readData(text, summary):
  print("Reading lines...")
  input_lang = Lang(text)
  output_lang = Lang(summary)
  pairs = [[text[i],summary[i]] for i in range(len(text))]
  return (input_lang, output_lang, pairs)

def prepareData(tlist, slist):
  input_lang, output_lang, pairs = readData(tlist, slist)
  print("Read %s sentence pairs" % len(pairs))
  print("Counting words...")  
  [input_lang.addSentence(t) for t in tlist]
  [output_lang.addSentence(s) for s in slist] 
  return input_lang, output_lang, pairs 

In [66]:
input, output, pairs = prepareData(X,Y)

Reading lines...
Read 197912 sentence pairs
Counting words...
